In [ ]:
# Make a data container for the yearly tiles with better naming
import arcpy
import os

out_folder_path = "D:\\Artikler\\KartOgPlan2024\\Data"
out_name = "GLC_FCS30b.gdb"
arcpy.management.CreateFileGDB(out_folder_path, out_name)

arcpy.env.workspace = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30.gdb"
rasters = arcpy.ListRasters("*")
# Loop through each raster
for r in rasters:
    r_name = os.path.splitext(r)[0]  # Get the filename without extension
    print(r_name)
    cr_name = f"{r_name[0:8]}_{r_name[41:len(r_name)]}"
    print(cr_name)
    out_rasterdataset = f"D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30b.gdb\\{cr_name}"
    arcpy.management.CopyRaster(r, out_rasterdataset)

In [ ]:
# Make a data container for the 1-yearly representations
out_folder_path = "D:\\Artikler\\KartOgPlan2024\\Data"
out_name = "GLC_FCS30_2000p.gdb"
arcpy.management.CreateFileGDB(out_folder_path, out_name)

In [ ]:
# extracting bands from multiband rasters and create single band rasters - for the 1-years representations
import arcpy
import os

arcpy.env.addOutputsToMap = False 

# Set the workspace
arcpy.env.workspace = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_AnnualMap"
output_gdb = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30_2000p.gdb"

# List of years corresponding to the bands
years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

# List all rasters with TIF format
rasters = arcpy.ListRasters("*", "TIF")

# Loop through each raster
for r in rasters:
    # Get raster properties
    raster = arcpy.Raster(r)
    band_count = raster.bandCount
    
    # Extract tile information from the raster name
    r_name = os.path.splitext(r)[0]  # Get the filename without extension
    tile = r_name[20:len(r_name) - 7]
    # Loop through each band and extract it to the geodatabase
    for band_num in range(1, band_count + 1):
        # Define the output name using the corresponding year and tile
        year = years[band_num - 1]  # Get the year for the band
        output_band = f"GLC_{year}_{tile}"  # Name it as GLC_1985_tile, GLC_1990_tile, etc.
        output_band_path = f"{output_gdb}\\{output_band}"

        # Extract the band and save it as a separate raster
        arcpy.management.CopyRaster(f"{r}\\Band_{band_num}", output_band_path, format="GRID")

        # Print progress for each band
        print(f"Extracted Band {band_num} (Year: {year}) to {output_band_path}")

In [ ]:
# working with the 1-years representations - mosaic to new raster
import arcpy
import os

output_gdb = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30mos.gdb"
arcpy.env.workspace = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30_2000p.gdb"

years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
for year in years:
    print(year)
    rasters = arcpy.ListRasters(f"*{year}*")
    output_location = output_gdb
    raster_dataset_name_with_extension = f"GLC_{year}"
    coordinate_system_for_the_raster = 4326
    pixel_type = "8_BIT_UNSIGNED"
    cellsize = 0.00026949458523585663
    number_of_bands = 1
    mosaic_method = "LAST"
    mosaic_colormap_mode = "FIRST"
    arcpy.management.MosaicToNewRaster(rasters, output_location, raster_dataset_name_with_extension, 
                                       coordinate_system_for_the_raster, pixel_type, cellsize, 
                                       number_of_bands, mosaic_method, mosaic_colormap_mode)

In [ ]:
# Project the data to UTM
# trouble with 2005 - done manually
arcpy.env.addOutputsToMap = False 
arcpy.env.workspace = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30mos.gdb"
output_gdb = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30mos_utm.gdb"
# List all rasters
rasters = arcpy.ListRasters("*")
# Loop through each raster
for r in rasters:
    print(r)
    out_raster = f"{output_gdb}\\{r}_utm"
    out_coor_system = 25833
    resampling_type = "NEAREST"
    cell_size = 30
    geographic_transform = "ETRS_1989_To_WGS_1984"
    in_coor_system = 4326
    arcpy.management.ProjectRaster(r, out_raster, out_coor_system, resampling_type, cell_size, geographic_transform, "", in_coor_system)

In [ ]:
# Make a Clip Raster
out_folder_path = "D:\\Artikler\\KartOgPlan2024\\Data"
out_name = "Geodata.gdb"
arcpy.management.CreateFileGDB(out_folder_path, out_name)
in_dataset = "D:\\Artikler\\KartOgPlan2024\\Data\\Norge_gcs.shp"
out_dataset = "D:\\Artikler\\KartOgPlan2024\\Data\\Geodata.gdb\\fcNorge"
arcpy.management.Project(in_dataset, out_dataset, 25833)
arcpy.env.snapRaster = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30mos_utm.gdb\\GLC_1985_utm"
rNorge = "D:\\Artikler\\KartOgPlan2024\\Data\\Geodata.gdb\\rNorge_utm"
arcpy.conversion.PolygonToRaster(out_dataset, 'Norge', rNorge, "MAXIMUM_AREA", "", 30)

In [ ]:
# Make a container for the finished rasters
out_folder_path = "D:\\Artikler\\KartOgPlan2024\\Data"
out_name = "GLC_FCS30_utm33.gdb"
arcpy.management.CreateFileGDB(out_folder_path, out_name)

In [ ]:
# clip the rasters
# trouble with 2005 - done manually
arcpy.env.addOutputsToMap = False 
arcpy.env.workspace = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30mos_utm.gdb"
output_gdb = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30_utm33.gdb"
arcpy.env.snapRaster = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30mos_utm.gdb\\GLC_1985_utm"
in_raster_or_constant2 = "D:\\Artikler\\KartOgPlan2024\\Data\\Geodata.gdb\\rNorge_utm"
# List all rasters
rasters = arcpy.ListRasters("*")
# Loop through each raster
for r in rasters:
    print(r)
    out_raster = f"{output_gdb}\\{r}"
    arcpy.ddd.Times(r, in_raster_or_constant2, out_raster)

In [ ]:
# Make a container for the reclassified rasters ...
out_folder_path = "D:\\Artikler\\KartOgPlan2024\\Data"
out_name = "GLC_FCS30_2cl.gdb"
arcpy.management.CreateFileGDB(out_folder_path, out_name)

In [ ]:
# Reclassify rasters
arcpy.env.addOutputsToMap = False 
out_gdg = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30_2cl.gdb"
arcpy.env.workspace = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30_utm33.gdb"
# List all rasters
rasters = arcpy.ListRasters("*")
# Loop through each raster
for r in rasters:
    print(r)
    remap = "0 188 0; 189 191 1; 192 250 0"
    out_raster = f"{out_gdg}\\{r}"
    arcpy.ddd.Reclassify(r, "Value", remap, out_raster)

In [ ]:
# Make a container for the zonal stat tables ...
out_folder_path = "D:\\Artikler\\KartOgPlan2024\\Data"
out_name = "ImpermTables.gdb"
arcpy.management.CreateFileGDB(out_folder_path, out_name)
# make a copy of the mun layer, simplify it and add iKomNr
in_features = "D:\\Artikler\\KartOgPlan2024\\Data\\Basisdata_0000_Norge_25833_Kommuner_FGDB.gdb\\kommune"
fcKom = "D:\\Artikler\\KartOgPlan2024\\Data\\Geodata.gdb\\fcKom"
arcpy.management.CopyFeatures(in_features, fcKom)
arcpy.management.CalculateField(fcKom, 'iKomNr', "!kommunenummer!", "PYTHON", "", "LONG")
keep_fields = ['iKomNr', 'kommunenummer', 'kommunenavn']
arcpy.management.DeleteField(fcKom, keep_fields, "KEEP_FIELDS")

In [ ]:
# run zonal statistics ...
import arcpy
from arcpy.sa import *
arcpy.env.addOutputsToMap = False
out_gdb = "D:\\Artikler\\KartOgPlan2024\\Data\\ImpermTables.gdb"

arcpy.env.workspace = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30_2cl.gdb"
inZoneData = "D:\\Artikler\\KartOgPlan2024\\Data\\Geodata.gdb\\fcKom"
zoneField = "iKomNr"
# List all rasters
rasters = arcpy.ListRasters("*")
# Loop through each raster
for r in rasters:
    year = int(r[4:8])
    print(year)
    inValueRaster = r
    outTable = f"{out_gdb}\\tbl{r}"
    outZSaT = ZonalStatisticsAsTable(inZoneData, zoneField, inValueRaster, outTable, "DATA", "SUM")

In [ ]:
# convert from unit pixels to unit square km
fc_pxl = "D:\\Artikler\\KartOgPlan2024\\Data\\Geodata.gdb\\fcKomGLC"
fc_km2 = "D:\\Artikler\\KartOgPlan2024\\Data\\Geodata.gdb\\fcKomGLCkm2"
arcpy.management.CopyFeatures(fc_pxl, fc_km2)
for year in range(2000, 2051):
    txtYear = str(year)
    if year < 2023: 
        old_field_name = f"ImpArea_{txtYear}"
        new_field_name = f"ImpAreaKm2_{txtYear}"
    elif year > 2022: 
        old_field_name = f"eImpAreal_{txtYear}"
        new_field_name = f"eImpAreaKm2_{txtYear}"
    arcpy.management.AddField(fc_km2, new_field_name, "DOUBLE")
    fields = [old_field_name, new_field_name]
    with arcpy.da.UpdateCursor(fc_km2, fields) as cursor:
        for row in cursor:
            row[1] = (row[0] * 900)/1000000
            cursor.updateRow(row)

In [ ]:
# clean up - delete fields with pixels as unit
fc = "D:\\Artikler\\KartOgPlan2024\\Data\\Geodata.gdb\\fcKomGLCkm2"
# Get a list of all fields in the feature class
all_fields = arcpy.ListFields(fc)  
# Filter fields that contain the year in their name
year_fields = [field.name for field in all_fields if 'ImpArea' in field.name]
km2_fields = [field.name for field in all_fields if 'Km2' in field.name]
year_fields_minus_km2 = [field for field in year_fields if field not in km2_fields]
arcpy.management.DeleteField(fc, year_fields_minus_km2)

# Generate change variables

In [ ]:
# Change in population
fc = "D:\\Artikler\\KartOgPlan2024\\Data\\Data2Trond.gdb\\fcKomPop"
arcpy.management.CalculateField(fc, 'bef00_22', "((!bef2000! - !bef2022!) / !bef2000!) * 100", "PYTHON", "", "DOUBLE")
arcpy.management.CalculateField(fc, 'bef23_50', "((!est2050! - !bef2023!) / !bef2023!) * 100", "PYTHON", "", "DOUBLE")
arcpy.management.CalculateField(fc, 'bef00_50', "((!est2050! - !bef2000!) / !bef2000!) * 100", "PYTHON", "", "DOUBLE")

In [ ]:
# Need to handle division by zero
fc = "D:\\Artikler\\KartOgPlan2024\\Data\\Data2Trond.gdb\\fcKomGLCkm2"
arcpy.management.AddField(fc, 'ImpArea00_22', "DOUBLE")
arcpy.management.AddField(fc, 'ImpArea23_50', "DOUBLE")
fields = ['ImpAreaKm2_2000', 'ImpAreaKm2_2022', 'ImpArea00_22']
with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        if not row[0] == 0:
            row[2] = ((row[1] - row[0])/row[0]) * 100
        else:
            row[2] = 0
        cursor.updateRow(row)
        
fields = ['eImpAreaKm2_2023', 'eImpAreaKm2_2050', 'ImpArea23_50']
with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        if not row[0] == 0:
            row[2] = ((row[1] - row[0])/row[0]) * 100
        else:
            row[2] = 0
        cursor.updateRow(row)

In [ ]:
# Need to handle division by zero
fc = "D:\\Artikler\\KartOgPlan2024\\Data\\Data2Trond.gdb\\fcKomGLCkm2"
arcpy.management.AddField(fc, 'ImpArea00_50', "DOUBLE")
fields = ['ImpAreaKm2_2000', 'eImpAreaKm2_2050', 'ImpArea00_50']
with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        if not row[0] == 0:
            row[2] = ((row[1] - row[0])/row[0]) * 100
        else:
            row[2] = 0
        cursor.updateRow(row)

In [ ]:
# Join fields to new main table
fcClean = "D:\\Prosjekter\\Arendalsuka_2024\\Data\\NewOldMunNr.gdb\\fcKom_LU"
fc = "D:\\Artikler\\KartOgPlan2024\\Data\\Data2Trond.gdb\\fcPopImpArea"
arcpy.management.CopyFeatures(fcClean, fc)
drop_field = ['iKomNr2023', 'txtFylke']
arcpy.management.DeleteField(fc, drop_field)
fcImpArea = "D:\\Artikler\\KartOgPlan2024\\Data\\Data2Trond.gdb\\fcKomGLCkm2"
fields = ['ImpArea00_22', 'ImpArea23_50']
arcpy.management.JoinField(fc, 'iKomNr', fcImpArea, 'iKomNr', fields)
arcpy.management.JoinField(fc, 'iKomNr', fcImpArea, 'iKomNr', 'ImpArea00_50')
fcPop = "D:\\Artikler\\KartOgPlan2024\\Data\\Data2Trond.gdb\\fcKomPop"
fields = ['bef00_22', 'bef23_50']
arcpy.management.JoinField(fc, 'iKomNr', fcPop, 'iKomNr', fields)
arcpy.management.JoinField(fc, 'iKomNr', fcPop, 'iKomNr', 'bef00_50')

# CONVERT TO TIFF

In [ ]:
import arcpy
import os
gdb = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30_utm33.gdb"
mappe = "D:\\Artikler\\KartOgPlan2024\\Data\\TIFF"
os.mkdir(mappe)
#loop through the rasters in the gdb and convert them to TIFF
arcpy.env.workspace = gdb
rasters = arcpy.ListRasters("*", "GRID")
for raster in rasters:
    print(raster)
    arcpy.conversion.RasterToOtherFormat(raster, mappe, "TIFF")

In [ ]:
# impermeable areas
import arcpy
import os
gdb = "D:\\Artikler\\KartOgPlan2024\\Data\\GLC_FCS30_2cl.gdb"
mappe = "D:\\Artikler\\KartOgPlan2024\\Data\\TIFF_Impermeable"
os.mkdir(mappe)
#loop through the rasters in the gdb and convert them to TIFF
arcpy.env.workspace = gdb
rasters = arcpy.ListRasters("*", "GRID")
for raster in rasters:
    print(raster)
    arcpy.conversion.RasterToOtherFormat(raster, mappe, "TIFF")